### Ce Notebook permet de convertir les fichiers d'entrainement Spacy2 en fichiers d'entrainement binaires Spacy3


- Ce notebook permet aussi suite à l'entrainement qui se fait dans un terminal de charger le modèle enregistré et d'évaluer sur les titres des recettes

In [5]:
import spacy
from spacy.tokens import DocBin
import json
from tqdm import tqdm
import random

In [28]:
#!pip install srsly

In [6]:
# Méthode pour lire un fichier json
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f)
    return (data)

### Chargement des data train/test/eval pour conversion en .spacy

In [30]:

ingr_train = load_data("data/ingredients_training_data2.json")

In [33]:
print('Dimension du data set : ',len(ingr_train),'  pour le train \n\n')

Dimension du data set :  144443   pour le train 




In [34]:
 print (ingr_train[0])

['mélang épic tandoor épic indien roug lait yaourt grec huil jus citron', {'entities': [[7, 11, 'INGREDIENT'], [12, 19, 'INGREDIENT'], [20, 24, 'INGREDIENT'], [25, 31, 'INGREDIENT'], [37, 41, 'INGREDIENT'], [42, 48, 'INGREDIENT'], [49, 53, 'INGREDIENT'], [54, 58, 'INGREDIENT'], [59, 62, 'INGREDIENT'], [63, 69, 'INGREDIENT']]}]


In [61]:
ingr_valid = load_data("data/ingredients_test_data2.json")
print('Dimension du data set : ',len(ingr_valid),'  pour le test \n\n')

Dimension du data set :  35568   pour le test 




In [62]:
ingrTitr_valid = load_data("data/ingredientsTitre_test_data2.json") 
print('Dimension du data set : ',len(ingrTitr_valid),'  pour l\'éval sur les titres \n\n')

Dimension du data set :  21648   pour l'éval sur les titres 




In [37]:
# Méthode pour conversion du format Spacy2 au format Spacy 3

nlp = spacy.blank("en")
def create_training(TRAIN_DATA):
    db = DocBin()
    for text, annot in tqdm(TRAIN_DATA): 
        doc = nlp.make_doc(text)
        ents = []
        for start, end, label in annot["entities"]:
            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None:
                print ("Skipping entity")
            else:
                ents.append(span)
        doc.ents = ents
        db.add(doc)
    return (db)

In [38]:
ingr_train = create_training(ingr_train)
ingr_train.to_disk("./data/ingr_train.spacy")

100%|██████████| 144443/144443 [03:58<00:00, 605.66it/s]


In [42]:
 
ingr_valid = create_training(ingr_valid)
ingr_valid.to_disk("./data/ingr_valid.spacy")

100%|██████████| 35568/35568 [00:50<00:00, 707.91it/s]


In [44]:

ingrTire_eval = create_training(ingrTitr_valid)
ingrTire_eval.to_disk("./data/ingrTitre_eval.spacy")

100%|██████████| 21648/21648 [00:32<00:00, 667.29it/s]


 ### On charge le meilleur modèle sauvegardé

In [45]:
from spacy.ml.models.tok2vec import MultiHashEmbed

In [46]:
#load the best model
nlp1 = spacy.load(R".\output\model-best") 

In [55]:
doc=nlp1('lasagn végétarien facil  terrin foi porc  lapin vin blanc cocott  escalop lucullus cocott  tart oignon carr coq vin facil')

In [56]:
spacy.displacy.render(doc, style="ent", jupyter=True)

In [57]:
# Prédiction
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

### On évalue: on visualise les éléments taggués comme ingrédients 

In [60]:
for txt in ingrTitr_valid:   
    selected_text = predict_entities(txt[0],nlp1)
    print(selected_text)

rc
viand
sucr
quino
moul
salad
riz
pois
pen
encornet
cuiss
conf
agneau
rôt
tart
steak
courget
tart
pan
tart
soup
pât
farfall
salad
pât
tart
thon
gâteau
salad
tart
dor
oseil
semoul
poulet
saumon
souffl
poulet
saumon
chocolat
agneau
poulet
bavarois
muffin
poulet
brochet
cook
biscuit
charlott
frambois
galet
tart
poisson
riz
soup
dind
salad
pissenlit
clémentin
curry
tart
pizz
amand
riz
gâteau
porc
terrin
filet
poulet
tart
agneau
cook
porc
saindoux
quich
asiat
magret
brioch
tart
noix
mouss
tart
pomm
mouss
flan
tart
poulet
avocat
quenel
macaron
sauc
fruit
moul
compot
rôt
pât
tart
épinard
lapin
pudding
salad
veau
cak
tartar
pain
pain
truit
flan
souffl
crevet
glac
charlott
frambois
tart
cak
boeuf
pain
pizz
fruit
rôt
pomm
gâteau
crem
nem
tart
crem
spaghet
quich
canet
glac
escalop
soup
filet
fruit
crem
foi
spéculoos
tartin
tart
papay
bavarois
magret
gâteau
tartin
choux
madelein
filet
fan
gâteau
yaourt
potiron
volaill
yaourt
pât
cuiss
jambon
dind
gâteau
chocolat
colin
cak
tart
pen
jambon
risotto
